In [1]:
import pandas as pd
import psycopg2
from rbbm_src.dc_src.DCRepair import (eq_op, 
 non_symetric_op,
#  dc_tuple_violation_template_targeted_t1,
#  dc_tuple_violation_template_targeted_t2
)
import re
from rbbm_src.dc_src.src.classes import parse_rule_to_where_clause
from string import Template

In [2]:
conn=psycopg2.connect('dbname=label user=postgres port=5433')
dc_tuple_violation_template_targeted_t1=\
Template("SELECT DISTINCT $t1_desc,$t2_desc FROM $table t1, $table t2 WHERE $dc_desc")
dc_tuple_violation_template_targeted_t2=\
Template("SELECT DISTINCT $t1_desc,$t2_desc FROM $table t1, $table t2 WHERE $dc_desc")

In [3]:
cols_q = f"select * from tax_full limit 1"
df_row = pd.read_sql(cols_q, conn)

In [4]:
cols=list(df_row)

In [5]:
cols

['fname',
 'lname',
 'gender',
 'areacode',
 'phone',
 'city',
 'state',
 'zip',
 'maritalstatus',
 'haschild',
 'salary',
 'rate',
 'singleexemp',
 'marriedexemp',
 'childexemp',
 'id']

In [6]:
def construct_query_for_violation(role, dc_text, target_table, targeted, cols): 
    predicates = dc_text.split('&')
#     clause = parse_rule_to_where_clause(dc_text)
    constants=[]
    # print(f"t_interest:{t_interest}")
    need_tid=True 
    # if the constraint only has equals, we need to add an artificial
    # key (_tid_) to differentiate tuples in violation with the tuple it
    # self
    for pred in predicates[2:]:
        if(not eq_op.search(pred)):
            need_tid=False
        attr = re.search(r't[1|2]\.([-\w]+)', pred).group(1).lower()
        # print(attr)
#         constants.append(f'{role}.{attr}=\'{t_interest[attr]}\'')
    constants_clause = ' AND '.join(constants)
    # print(f"dc_text:{dc_text}")
    t1_cols=', '.join([f't1.{x} as t1_{x}' for x in cols])
    t2_cols=', '.join([f't2.{x} as t2_{x}' for x in cols])
    
    if(role=='t1'):
        template=dc_tuple_violation_template_targeted_t1
    else:
        template=dc_tuple_violation_template_targeted_t2
    if(targeted):
        r_q  = template.substitute(t1_desc=t1_cols, t2_desc=t2_cols,
                                   table=target_table, dc_desc=parse_rule_to_where_clause(dc_text),
                                           tuple_desc=constants_clause)

    else:
        r_q  = template.substitute(t1_desc=t1_cols, t2_desc=t2_cols,
                                   table=target_table, dc_desc=parse_rule_to_where_clause(dc_text))

        r_q+=f" AND t1.id!=t2.id"
#     print(r_q)

    return r_q

def find_tuples_in_violation(conn, dc_text, target_table, cols, targeted=False):
    if(non_symetric_op.search(dc_text)):
        q1= construct_query_for_violation('t1', dc_text, target_table, targeted, cols)
        q2 = construct_query_for_violation('t2', dc_text, target_table, targeted, cols)
        res = {'t1': pd.read_sql(q1, conn).to_dict('records'), 
        't2': pd.read_sql(q2, conn).to_dict('records')}
    else:
        q1= construct_query_for_violation('t1', dc_text, target_table, targeted, cols)
        print(q1)
        res = {'t1':pd.read_sql(q1, conn).to_dict('records')}
        # print(res)
    return res 

In [7]:
dc_file='tax_precise_dcs.txt'
table_name='tax_sample'
res=[]
rule_texts=[]
try:
    with open(dc_file, "r") as file:
        for line in file:
            rule_texts.append(line.strip())
            # rules_from_line = [(table_name, x) for x in convert_dc_to_muse_rule(line, 'adult', 't1')]
#             rules_from_line = [(table_name, x) for x in convert_dc_to_muse_rule(line, 'flights_new', 't1')]
#             res.extend(rules_from_line)
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("Error reading the file.")


In [8]:
print(len(rule_texts))

30053


In [9]:
rule_texts

['t1&t2&LTE(t1.Zip,t2.Zip)&LTE(t1.Rate,t2.Rate)&GTE(t1.Salary,t2.Salary)&GTE(t1.ChildExemp,t2.ChildExemp)&IQ(t1.State,t2.State)&IQ(t1.MarriedExemp,t2.MarriedExemp)&GTE(t1.Phone,t2.Phone)&EQ(t1.City,t2.City)',
 't1&t2&LTE(t1.Phone,t2.Phone)&EQ(t1.AreaCode,t2.AreaCode)&GTE(t1.Salary,t2.Salary)&EQ(t1.MarriedExemp,t2.MarriedExemp)&EQ(t1.ChildExemp,t2.ChildExemp)&IQ(t1.MaritalStatus,t2.MaritalStatus)',
 't1&t2&IQ(t1.Rate,t2.Rate)&GTE(t1.Salary,t2.Salary)&GT(t1.ChildExemp,t2.ChildExemp)&GT(t1.MarriedExemp,t2.MarriedExemp)&GTE(t1.Phone,t2.Phone)&EQ(t1.City,t2.City)',
 't1&t2&LTE(t1.Zip,t2.Zip)&LTE(t1.SingleExemp,t2.SingleExemp)&LTE(t1.MarriedExemp,t2.MarriedExemp)&LTE(t1.AreaCode,t2.AreaCode)&GTE(t1.ChildExemp,t2.ChildExemp)&EQ(t1.Phone,t2.Phone)&IQ(t1.HasChild,t2.HasChild)',
 't1&t2&LTE(t1.Zip,t2.Zip)&IQ(t1.SingleExemp,t2.SingleExemp)&GTE(t1.Salary,t2.Salary)&EQ(t1.FName,t2.FName)&EQ(t1.MarriedExemp,t2.MarriedExemp)&LTE(t1.Rate,t2.Rate)&EQ(t1.HasChild,t2.HasChild)',
 't1&t2&EQ(t1.Rate,t2.Rat

In [20]:
dirty_tuples = set()

In [25]:
rules_with_stats={r:set([]) for r in rule_texts[:10]}

In [26]:
for r in rule_texts[:10]:
    res = find_tuples_in_violation(conn, r, table_name, cols)
    print(f"rule: {r}")
    for k in res:
        for pair in res[k]:
#             if(pair['t1_is_dirty']!=pair['t2_is_dirty']):
            # dirty_tuples.add(pair['t1_id'])
            # dirty_tuples.add(pair['t2_id'])
        # len_cc = len([x for x in res[k] if (x['t1_is_dirty']==False and x['t2_is_dirty']==False)])
        # len_non_cc = len([x for x in res[k] if not (x['t1_is_dirty']==False and x['t2_is_dirty']==False)])
        # len_dd = len([x for x in res[k] if (x['t1_is_dirty']==True and x['t2_is_dirty']==True)])
        # len_dc = len([x for x in res[k] if ((x['t1_is_dirty']==True and x['t2_is_dirty']==False) or \
        #             (x['t1_is_dirty']==False and x['t2_is_dirty']==True))])
            rules_with_stats[r].add(pair['t1_id'])
            rules_with_stats[r].add(pair['t2_id'])
        # rules_with_stats[r].add(pair['t1_id'])
        # print(f"number of ccs:{len_cc}, number of non ccs: {len_non_cc}, number of dds: {len_dd}, number of dcs:{len_dc}")
#     break
    print('\n')

rule: t1&t2&LTE(t1.Zip,t2.Zip)&LTE(t1.Rate,t2.Rate)&GTE(t1.Salary,t2.Salary)&GTE(t1.ChildExemp,t2.ChildExemp)&IQ(t1.State,t2.State)&IQ(t1.MarriedExemp,t2.MarriedExemp)&GTE(t1.Phone,t2.Phone)&EQ(t1.City,t2.City)


rule: t1&t2&LTE(t1.Phone,t2.Phone)&EQ(t1.AreaCode,t2.AreaCode)&GTE(t1.Salary,t2.Salary)&EQ(t1.MarriedExemp,t2.MarriedExemp)&EQ(t1.ChildExemp,t2.ChildExemp)&IQ(t1.MaritalStatus,t2.MaritalStatus)


rule: t1&t2&IQ(t1.Rate,t2.Rate)&GTE(t1.Salary,t2.Salary)&GT(t1.ChildExemp,t2.ChildExemp)&GT(t1.MarriedExemp,t2.MarriedExemp)&GTE(t1.Phone,t2.Phone)&EQ(t1.City,t2.City)


rule: t1&t2&LTE(t1.Zip,t2.Zip)&LTE(t1.SingleExemp,t2.SingleExemp)&LTE(t1.MarriedExemp,t2.MarriedExemp)&LTE(t1.AreaCode,t2.AreaCode)&GTE(t1.ChildExemp,t2.ChildExemp)&EQ(t1.Phone,t2.Phone)&IQ(t1.HasChild,t2.HasChild)


rule: t1&t2&LTE(t1.Zip,t2.Zip)&IQ(t1.SingleExemp,t2.SingleExemp)&GTE(t1.Salary,t2.Salary)&EQ(t1.FName,t2.FName)&EQ(t1.MarriedExemp,t2.MarriedExemp)&LTE(t1.Rate,t2.Rate)&EQ(t1.HasChild,t2.HasChild)


SELEC

In [27]:
for r,v in rules_with_stats.items():
    print(r)
    print(len(v))
    print('\n')

t1&t2&LTE(t1.Zip,t2.Zip)&LTE(t1.Rate,t2.Rate)&GTE(t1.Salary,t2.Salary)&GTE(t1.ChildExemp,t2.ChildExemp)&IQ(t1.State,t2.State)&IQ(t1.MarriedExemp,t2.MarriedExemp)&GTE(t1.Phone,t2.Phone)&EQ(t1.City,t2.City)
298


t1&t2&LTE(t1.Phone,t2.Phone)&EQ(t1.AreaCode,t2.AreaCode)&GTE(t1.Salary,t2.Salary)&EQ(t1.MarriedExemp,t2.MarriedExemp)&EQ(t1.ChildExemp,t2.ChildExemp)&IQ(t1.MaritalStatus,t2.MaritalStatus)
2777


t1&t2&IQ(t1.Rate,t2.Rate)&GTE(t1.Salary,t2.Salary)&GT(t1.ChildExemp,t2.ChildExemp)&GT(t1.MarriedExemp,t2.MarriedExemp)&GTE(t1.Phone,t2.Phone)&EQ(t1.City,t2.City)
693


t1&t2&LTE(t1.Zip,t2.Zip)&LTE(t1.SingleExemp,t2.SingleExemp)&LTE(t1.MarriedExemp,t2.MarriedExemp)&LTE(t1.AreaCode,t2.AreaCode)&GTE(t1.ChildExemp,t2.ChildExemp)&EQ(t1.Phone,t2.Phone)&IQ(t1.HasChild,t2.HasChild)
7485


t1&t2&LTE(t1.Zip,t2.Zip)&IQ(t1.SingleExemp,t2.SingleExemp)&GTE(t1.Salary,t2.Salary)&EQ(t1.FName,t2.FName)&EQ(t1.MarriedExemp,t2.MarriedExemp)&LTE(t1.Rate,t2.Rate)&EQ(t1.HasChild,t2.HasChild)
341


t1&t2&EQ(t1.R

In [16]:
len(dirty_tuples)

8431

In [ ]:
dirty_tuples

In [ ]:
len(dirty_tuples)

In [ ]:
rules_with_stats

In [ ]:
sorted(rules_with_stats, key=lambda k: rules_with_stats[k]['len_cc'], reverse=True)

In [ ]:
list({k:v for k,v in rules_with_stats.items() if v['len_cc']==0})